## Identify WDPAs in each gadm1


This notebook was created in July 2022 to modify the WDPA tables associated with the precalculated geometries shown on the AOI section. As part of our tasks to enrich the AOI section, we need to include a tooltip that provides some information about the protected areas when the user clicks on them. We realized that there was a field missing on the old tables, the STATUS_ field that refers to the year in which the STATUS was established.

We have reproduced the creation of these tables to be able to integrate this field. It could have been possible to do it using only the MOL_ID present in the WDPA original table and on the WDPA precalculated tables. However, for gadm0, gadm1 and future places we decided to repeat the joins to make sure they rendered the same results (as the first version of this table was done differently or by different team members). In the case of gadm0, for instance, instead of associating to each country those WDPAs that intersect their boundaries (as it was done in the past), we kept the entire dataset, using the ISO3 or GID_0 field to determine to which country a given WDPA is related to.

## Table of contents
1. [First steps](#first)
    1. [Import packages](#packages)
    2. [Connect to ESRI](#esri)
    3. [Utils](#utils)
    4. [Import layers and tables](#layers)
2. [Preprocessing](#preprocessing)
    1. [Update names of gadm0 and gadm1](#update)
    2. [Select relevant columns](#columns)
3. [Create tables](#tables)
    1. [Gadm0](#gadm0)
    2. [Gadm1](#gadm1)
    3. [Places](#places)
    4. [WDPA table with precalculated](#wdpa)
4. [Publish layers with API](#publish)


----
<a id='first'></a>
## First steps 

<a id='packages'></a>
### Import packages

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
import requests as re
from copy import deepcopy

<a id='esri'></a>
### Connect to arcgis online with credentials saved in .env file

In [2]:
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)

<a id='utils'></a>
### Utils

In [33]:
# Function to publish files as feature services or hosted tables

def publishCSVasFS(csv_file,csvName, gis, aol_folder_name, sharing, table): # sharing = 'None', 'groups', 'everyone' # table = 'yes', 'no'
    try:
        
        if gis.content.is_service_name_available(csvName, "featureService"):
            print(f"Service name {csvName} is available")
            #csv_file = f'./{csvName}.csv'
            csv_item = gis.content.add({}, csv_file)
            #csv_lyr = csv_item.publish(None,  {'CountryCode' : 'countryname'})
            if table == 'yes':
                csv_lyr = csv_item.publish(publish_parameters={'locationType':'none', 'type':'csv', 'name': csvName})
            else:
                csv_lyr = csv_item.publish()
                flayer_collection = FeatureLayerCollection.fromitem(csv_lyr)
                searched_flayer = flayer_collection.layers[0] 
                nber_features = searched_flayer.query(return_count_only=True)
                print(f"The service {csvName} has been published. The service has {nber_features} entries")
            print(f"Moving service {csvName} to {aol_folder_name} in ArcGIS Online...")
            # move both the csv_item and csv_lyr items into this new folder
            csv_item.move(aol_folder_name)
            csv_lyr.move(aol_folder_name)
            print(f"Service {csvName} has been moved to {aol_folder_name} in ArcGIS Online")
#             sharing, default is owner
#             csv_lyr.shared_with
            if sharing == 'None':
                csv_lyr.share(everyone=False, org=False, groups=None, allow_members_to_edit=False)
            if sharing == 'groups':
            # the group id is needed to share with agroup
                csv_lyr.share(everyone=False, org=False, groups=[group_id], allow_members_to_edit=False)
            if sharing == 'everyone':
                csv_lyr.share(everyone=True, org=False, groups=None, allow_members_to_edit=False)
            sharing_prop = csv_lyr.shared_with
            if sharing_prop['everyone']==True:
                print(f"shared with everyone")   
            else:
                print(f"not a public layer, for this layer to be used it has to be public or the urls have to be whitelisted")            
            #not allowing deleting
            csv_lyr.protect()
            print(f"{csvName}'s protection against deletion : {csv_lyr.protected}") 
            # it is possible to check the status with csv_item.content_status
            return csv_lyr.id
        else:
            print("The service name is not available, try overwritting, appending the data or a different service name")
    except Exception as e:
        print(e)

<a id='layers'></a>
### Import layers
Here we will import the layers that have to be updated. For gadm0, gadm1 and future places, the WDPA are given in a different table and the precalculated data is stored in feature layers with geometries. For the WDPAs, however, the precalculated data (biodiversity + contextual) is stored in a hosted table, while the geometries are given in 3 different feature layers with different degrees of simplification.

In [3]:
## Table of WDPA only terrestrial + coastal (MOL https://eowilson.maps.arcgis.com/home/item.html?id=852a88b71d0846e79979ac541c526b33#data)
wdpa = gpd.read_file('/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Inputs/WDPA_FILTERED_20210615_FILTERED_nomarine_with_oecm_wdpa/WDPA_FILTERED_20210615_FILTERED_nomarine_with_oecm_wdpa.shp')

In [4]:
# National boundaries (from https://eowilson.maps.arcgis.com/home/item.html?id=f11bd71ecc4c47aab7735b3cf3e21392)
gadm0 = gpd.read_file('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/Inputs/gadm36_level0_original_simple.shp')

In [5]:
# Subnational boundaries (from https://eowilson.maps.arcgis.com/home/item.html?id=fe214eeebd21493eb2782a7ce1466606)
gadm1 = gpd.read_file('/Users/sofia/Documents/HE_Data/Precalculated/gadm1/gadm36_level1/gadm36_level1.shp')

In [6]:
# Places for the HE future (from https://eowilson.maps.arcgis.com/home/item.html?id=b90850994e574a2cab153c7e679399f6 converted to EPSG 4326 in ArcGIS Pro)
places = gpd.read_file('/Users/sofia/Documents/HE_Data/PlacesHE/Inputs/Geometries/places_good_geom_prj/places_good_geom_prj.shp')

In [7]:
# WDPA table with biodiversity + contextual precalculated data
wdpa_table = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Outputs/wdpa_precal_with_contextual.csv')
wdpa_table.head()

,Unnamed: 0,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,...,mammals,reptiles,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum
0,0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...",[],107.0,Grassland,Cool Temperate Moist,0.0,0.0,100.00,0.0,2.110001
1,1,Ferndale,2,0.748492,307797.0,Scenic Reserve,Scenic Reserve,National,III,NZL,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",176.0,Forest,Warm Temperate Moist,0.0,0.0,0.00,0.0,1.315837
2,2,Broughton Bay,3,0.031907,307745.0,Scenic Reserve,Scenic Reserve,National,III,NZL,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6499,""per_global"":0.0,""per_aoi...",0.0,NaN,NaN,0.0,0.0,0.00,0.0,0.000000
3,3,Kaipupu Point,4,0.270855,307867.0,Scenic Reserve,Scenic Reserve,National,III,NZL,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",0.0,NaN,NaN,0.0,0.0,100.00,0.0,0.000000
4,4,Catlins Conservation Park,5,8.412168,303963.0,Stewardship Area,Stewardship Area,National,III,NZL,...,[],"[{""SliceNumber"":6163,""per_global"":0.01,""per_ao...",97.0,Forest,Cool Temperate Moist,0.0,0.0,83.33,0.0,3.103363


----
<a id='preprocessing'></a>
## Preprocessing

<a id='update'></a>
### Update names of gadm1 and gadm0 using gadm version 4.0
Because the gadm0 and gadm1 datasets we have correspond to version 3.6 and there are some countries/regions that have changed their name, we are going to update some administrative names to gadm version 4.0. 

In [8]:
gadm40 = gpd.read_file('/Users/sofia/Documents/HE_Data/gadm/gadm404-shp/gadm404.shp')

**gadm0**

In [9]:
gadm40_GID0 = gadm40[['GID_0', 'NAME_0']]
gadm40_GID0 = gadm40_GID0.groupby('GID_0')
gadm40_GID0 = gadm40_GID0.first()
gadm40_GID0 = gadm40_GID0.reset_index()
gadm40_GID0 = gadm40_GID0.rename(columns={'GID_0':'GID_0_new', 'NAME_0':'NAME_0_new'})
gadm40_GID0

,GID_0_new,NAME_0_new
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,AIA,Anguilla
4,ALA,Åland
...,...,...
258,Z09,Sang
259,ZAF,South Africa
260,ZMB,Zambia
261,ZNC,Northern Cyprus


In [10]:
gadm0 = pd.merge(gadm0, gadm40_GID0, how='left', left_on='GID_0', right_on='GID_0_new')
gadm0.head()

,GID_0,NAME_0,AREA_KM2,MOL_ID,InPoly_FID,SimPgnFlag,MaxSimpTol,MinSimpTol,geometry,GID_0_new,NAME_0_new
0,ABW,Aruba,1.819384e+02,1,1,0,0.005,0.005,"POLYGON ((-69.97820 12.46986, -70.02847 12.503...",ABW,Aruba
1,AFG,Afghanistan,6.438575e+05,2,2,0,0.005,0.005,"POLYGON ((68.52644 31.75435, 68.53852 31.75457...",AFG,Afghanistan
2,AGO,Angola,1.247422e+06,3,3,0,0.005,0.005,"MULTIPOLYGON (((11.73347 -16.67255, 11.74014 -...",AGO,Angola
3,AIA,Anguilla,8.330331e+01,4,4,0,0.005,0.005,"MULTIPOLYGON (((-63.42375 18.58903, -63.42847 ...",AIA,Anguilla
4,ALA,Åland,1.506261e+03,5,5,0,0.005,0.005,"MULTIPOLYGON (((21.32195 59.74986, 21.32472 59...",ALA,Åland


In [12]:
gadm2 = gadm0[gadm0.NAME_0!=gadm0.NAME_0_new]
gadm2[['NAME_0', 'NAME_0_new']]

,NAME_0,NAME_0_new
47,Republic of Congo,Republic of the Congo
51,Cape Verde,Cabo Verde
58,Czech Republic,Czechia
96,Hong Kong,NaN
135,Macao,NaN
144,Macedonia,North Macedonia
184,Palestina,Palestine
187,Reunion,Réunion
196,Saint Helena,"Saint Helena, Ascension and Tristan da Cunha"
248,Northern Cyprus,NaN


In [13]:
# Give to each country with NaN values in gadm40 the name they had in gadm36
gadm0.NAME_0_new.fillna(gadm0.NAME_0, inplace=True)

In [14]:
# Check the names that are different now and that will change
gadm2 = gadm0[gadm0.NAME_0!=gadm0.NAME_0_new]
gadm2[['NAME_0', 'NAME_0_new']]

,NAME_0,NAME_0_new
47,Republic of Congo,Republic of the Congo
51,Cape Verde,Cabo Verde
58,Czech Republic,Czechia
144,Macedonia,North Macedonia
184,Palestina,Palestine
187,Reunion,Réunion
196,Saint Helena,"Saint Helena, Ascension and Tristan da Cunha"


In [15]:
# Give to NAME_0 the new names
gadm0.NAME_0 = gadm0.NAME_0_new
gadm0 = gadm0.drop(columns={'NAME_0_new', 'GID_0_new'})
gadm0

,GID_0,NAME_0,AREA_KM2,MOL_ID,InPoly_FID,SimPgnFlag,MaxSimpTol,MinSimpTol,geometry
0,ABW,Aruba,1.819384e+02,1,1,0,0.005,0.005,"POLYGON ((-69.97820 12.46986, -70.02847 12.503..."
1,AFG,Afghanistan,6.438575e+05,2,2,0,0.005,0.005,"POLYGON ((68.52644 31.75435, 68.53852 31.75457..."
2,AGO,Angola,1.247422e+06,3,3,0,0.005,0.005,"MULTIPOLYGON (((11.73347 -16.67255, 11.74014 -..."
3,AIA,Anguilla,8.330331e+01,4,4,0,0.005,0.005,"MULTIPOLYGON (((-63.42375 18.58903, -63.42847 ..."
4,ALA,Åland,1.506261e+03,5,5,0,0.005,0.005,"MULTIPOLYGON (((21.32195 59.74986, 21.32472 59..."
...,...,...,...,...,...,...,...,...,...
250,XSP,Spratly Islands,1.874750e+00,251,251,0,0.005,0.005,"MULTIPOLYGON (((114.35472 11.45139, 114.35167 ..."
251,YEM,Yemen,4.522035e+05,252,252,0,0.005,0.005,"MULTIPOLYGON (((53.26931 12.12875, 53.31011 12..."
252,ZAF,South Africa,1.219710e+06,253,253,0,0.005,0.005,"MULTIPOLYGON (((19.42097 -34.67903, 19.42208 -..."
253,ZMB,Zambia,7.504919e+05,254,254,0,0.005,0.005,"POLYGON ((25.88753 -17.98407, 25.88284 -17.984..."


In [16]:
# Now Czech Republic is Czechia
gadm0.NAME_0[gadm0.GID_0=='CZE']

58    Czechia
Name: NAME_0, dtype: object

**gadm1**

In [17]:
gadm40_GID1 = gadm40[['GID_1', 'NAME_1']]
gadm40_GID1 = gadm40_GID1.groupby('GID_1')
gadm40_GID1 = gadm40_GID1.first()
gadm40_GID1 = gadm40_GID1.reset_index()
gadm40_GID1 = gadm40_GID1.rename(columns={'GID_1':'GID_1_new', 'NAME_1':'NAME_1_new'})
gadm40_GID1

,GID_1_new,NAME_1_new
0,?,?
1,AFG.10_1,Ghor
2,AFG.11_1,Hilmand
3,AFG.12_1,Hirat
4,AFG.13_1,Jawzjan
...,...,...
3651,ZWE.5_1,Mashonaland East
3652,ZWE.6_1,Mashonaland West
3653,ZWE.7_1,Masvingo
3654,ZWE.8_1,Matabeleland North


In [18]:
gadm40_GID1 = gadm40_GID1.rename(columns={'GID_1':'GID_1_new', 'NAME_1':'NAME_1_new'})

In [19]:
# Merge both datasets by GID
gadm1 = pd.merge(gadm1, gadm40_GID1, how='left', left_on='GID_1', right_on='GID_1_new')

In [20]:
# Give to regions with nan name the name they had in gadm36
gadm1.NAME_1_new.fillna(gadm1.NAME_1, inplace=True)

In [21]:
gadm1.NAME_1 = gadm1.NAME_1_new
gadm1 = gadm1.drop(columns={'NAME_1_new', 'GID_1_new'})

In [22]:
gadm1[gadm1.NAME_1=='Iğdır']

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,AREA_KM2,MOL_ID,geometry
3156,TUR,Turkey,TUR.38_1,Iğdır,None,None,Il,Province,None,TR.IG,3929.341409,3157,"POLYGON ((44.33969 40.03029, 44.34463 40.02792..."


<a id='columns'></a>
### Select relevant columns
Let's select the fields we want to keep in the WDPA dataset, and also relevant columns in the dataset with the precalculated data. Since the precalculated data is saved in feature layers, for these hosted table we do not need biodiversity and contextual data. In fact, we only need the names of the geometries and their MOL_ID. To be able to distinguish this unique ID (the MOL_ID) from the one in the WDPA table, we are going to change the names in gadm0 and gadm1 to `MOL_IDg`. In the future places layer, the unique ID is called `places`. In this way, in the resulting tables `MOL_ID` will always refer to the ID of the WDPA.

In [10]:
wdpa.columns

Index(['WDPAID', 'WDPA_PID', 'PA_DEF', 'NAME', 'ORIG_NA', 'DESIG', 'DESIG_E',
       'DESIG_T', 'IUCN_CA', 'INT_CRI', 'MARINE', 'REP_M_A', 'GIS_M_A',
       'REP_ARE', 'GIS_ARE', 'NO_TAKE', 'NO_TK_A', 'STATUS', 'STATUS_',
       'GOV_TYP', 'OWN_TYP', 'MANG_AU', 'MANG_PL', 'VERIF', 'METADAT',
       'SUB_LOC', 'PARENT_', 'ISO3', 'SUPP_IN', 'CONS_OB', 'SORTER',
       'WDPA_PID_h', 'WDPA_PID__', 'hash_vl', 'AREA_KM', 'MOL_ID', 'geometry'],
      dtype='object')

In [23]:
# Select relevant fields in WDPA dataset. Note that STATUS_ is included
wd = wdpa[['WDPA_PID','MOL_ID','ISO3','NAME','ORIG_NA','STATUS','STATUS_','GOV_TYP','DESIG','DESIG_E','DESIG_T','IUCN_CA','AREA_KM','geometry']] 

In [24]:
gadm0 = gadm0[['GID_0','NAME_0','MOL_ID']]
gadm0.rename(columns={'MOL_ID':'MOL_IDg'},inplace=True)

/opt/anaconda3/envs/vizz38/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [25]:
gadm1 = gadm1[['GID_0','NAME_0','NAME_1','MOL_ID','geometry']]
gadm1.rename(columns={'MOL_ID':'MOL_IDg'},inplace=True)

In [26]:
places = places[['GID_0', 'NAME_0', 'cluster', 'geometry']]
places.rename(columns={'cluster':'places'}, inplace=True)

---
<a id='tables'></a>
## Create tables

<a id='gadm0'></a>
### Gadm0

In this case, instead of doing a spatial join, we are going to keep the entire table (as it has the ISO3 field that determines to which country the WDPA belongs) and we are just going to add GID_0 and NAME_0 because those are fields used in the frontend.

In [42]:
g0_wd = pd.merge(wd, gadm0, how='left', left_on='ISO3', right_on='GID_0')
g0_wd.head()

,WDPA_PID,MOL_ID,ISO3,NAME,ORIG_NA,STATUS,STATUS_,GOV_TYP,DESIG,DESIG_E,DESIG_T,IUCN_CA,AREA_KM,geometry,GID_0,NAME_0,MOL_IDg
0,310492,1,NZL,Boulder Beach,Boulder Beach / WWF Block,Designated,NaN,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,1.136031,"MULTIPOLYGON (((170.62746 -45.89520, 170.62824...",NZL,New Zealand,171.0
1,307797,2,NZL,Ferndale,Ferndale,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.748492,"POLYGON ((173.97642 -41.19258, 173.97595 -41.1...",NZL,New Zealand,171.0
2,307745,3,NZL,Broughton Bay,Broughton Bay,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.031907,"POLYGON ((173.95691 -41.20976, 173.95580 -41.2...",NZL,New Zealand,171.0
3,307867,4,NZL,Kaipupu Point,Kaipupu Point,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.270855,"POLYGON ((174.00241 -41.27336, 174.00312 -41.2...",NZL,New Zealand,171.0
4,303963,5,NZL,Catlins Conservation Park,Catlins Conservation Park,Designated,NaN,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,8.412168,"MULTIPOLYGON (((169.30645 -46.57949, 169.30754...",NZL,New Zealand,171.0


In [43]:
g0_wd = g0_wd.drop(columns='geometry')
g0_wd.head()

,WDPA_PID,MOL_ID,ISO3,NAME,ORIG_NA,STATUS,STATUS_,GOV_TYP,DESIG,DESIG_E,DESIG_T,IUCN_CA,AREA_KM,GID_0,NAME_0,MOL_IDg
0,310492,1,NZL,Boulder Beach,Boulder Beach / WWF Block,Designated,NaN,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,1.136031,NZL,New Zealand,171.0
1,307797,2,NZL,Ferndale,Ferndale,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.748492,NZL,New Zealand,171.0
2,307745,3,NZL,Broughton Bay,Broughton Bay,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.031907,NZL,New Zealand,171.0
3,307867,4,NZL,Kaipupu Point,Kaipupu Point,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.270855,NZL,New Zealand,171.0
4,303963,5,NZL,Catlins Conservation Park,Catlins Conservation Park,Designated,NaN,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,8.412168,NZL,New Zealand,171.0


In [44]:
g0_wd.to_csv('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/wdpa_with_gadm0_updated.csv')

<a id='gadm1'></a>
### Gadm1

In [13]:
g1_wd = gpd.sjoin(gadm1,wd, how='right')

In [14]:
g1_wd.head()

,index_left,GID_0,NAME_0,NAME_1,MOL_IDg,WDPA_PID,MOL_ID,ISO3,NAME,ORIG_NA,STATUS,STATUS_,GOV_TYP,DESIG,DESIG_E,DESIG_T,IUCN_CA,AREA_KM,geometry
0,2214.0,NZL,New Zealand,Otago,2215.0,310492,1,NZL,Boulder Beach,Boulder Beach / WWF Block,Designated,NaN,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,1.136031,"MULTIPOLYGON (((170.62746 -45.89520, 170.62824..."
1,2210.0,NZL,New Zealand,Marlborough,2211.0,307797,2,NZL,Ferndale,Ferndale,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.748492,"POLYGON ((173.97642 -41.19258, 173.97595 -41.1..."
2,2210.0,NZL,New Zealand,Marlborough,2211.0,307745,3,NZL,Broughton Bay,Broughton Bay,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.031907,"POLYGON ((173.95691 -41.20976, 173.95580 -41.2..."
3,2210.0,NZL,New Zealand,Marlborough,2211.0,307867,4,NZL,Kaipupu Point,Kaipupu Point,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.270855,"POLYGON ((174.00241 -41.27336, 174.00312 -41.2..."
4,2214.0,NZL,New Zealand,Otago,2215.0,303963,5,NZL,Catlins Conservation Park,Catlins Conservation Park,Designated,NaN,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,8.412168,"MULTIPOLYGON (((169.30645 -46.57949, 169.30754..."


In [15]:
g1_wd = g1_wd.drop(columns=['index_left'])
g1_wd = g1_wd.drop(columns=['geometry'])
g1_wd.to_csv('/Users/sofia/Documents/HE_Data/Precalculated/gadm1/wdpa_with_gadm1_updated.csv')

<a id='places'></a>
### Future places

In [89]:
places

,GID_0,NAME_0,places,geometry
0,AFG,Afghanistan,1,"POLYGON ((70.31962 33.89062, 70.30704 33.88078..."
1,AFG,Afghanistan,2,"POLYGON ((70.82692 34.65551, 70.81682 34.66608..."
2,AFG,Afghanistan,3,"POLYGON ((68.82874 33.74859, 68.81840 33.75925..."
3,AFG,Afghanistan,4,"POLYGON ((67.83208 33.44323, 67.82402 33.45285..."
4,AFG,Afghanistan,5,"POLYGON ((67.58208 33.74502, 67.57402 33.75468..."
...,...,...,...,...
1916,ZWE,Zimbabwe,1921,"POLYGON ((25.83208 -18.15056, 25.82402 -18.142..."
1917,ZWE,Zimbabwe,1922,"POLYGON ((28.33208 -21.36464, 28.32402 -21.356..."
1918,ZWE,Zimbabwe,1923,"POLYGON ((25.25000 -17.70405, 25.26936 -17.704..."
1919,ZWE,Zimbabwe,1924,"POLYGON ((26.08208 -18.94826, 26.07402 -18.939..."


In [88]:
pl_wd = gpd.sjoin(places,wd, how='right')
pl_wd

,index_left,GID_0,NAME_0,places,WDPA_PID,MOL_ID,ISO3,NAME,ORIG_NA,STATUS,STATUS_,GOV_TYP,DESIG,DESIG_E,DESIG_T,IUCN_CA,AREA_KM,geometry
0,NaN,NaN,NaN,NaN,310492,1,NZL,Boulder Beach,Boulder Beach / WWF Block,Designated,NaN,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,1.136031,"MULTIPOLYGON (((170.62746 -45.89520, 170.62824..."
1,1150.0,NZL,New Zealand,1155.0,307797,2,NZL,Ferndale,Ferndale,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.748492,"POLYGON ((173.97642 -41.19258, 173.97595 -41.1..."
2,1150.0,NZL,New Zealand,1155.0,307745,3,NZL,Broughton Bay,Broughton Bay,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.031907,"POLYGON ((173.95691 -41.20976, 173.95580 -41.2..."
3,1150.0,NZL,New Zealand,1155.0,307867,4,NZL,Kaipupu Point,Kaipupu Point,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.270855,"POLYGON ((174.00241 -41.27336, 174.00312 -41.2..."
4,NaN,NaN,NaN,NaN,303963,5,NZL,Catlins Conservation Park,Catlins Conservation Park,Designated,NaN,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,8.412168,"MULTIPOLYGON (((169.30645 -46.57949, 169.30754..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217481,NaN,NaN,NaN,NaN,555716517,217482,PHL,Sablayan Fish Sanctuary,Sablayan Fish Sanctuary,Established,2000.0,Not Reported,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,0.000000,None
217482,NaN,NaN,NaN,NaN,555716527,217483,PHL,Gurion Marine Sanctuary,Gurion Marine Sanctuary,Established,2002.0,Not Reported,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,0.000000,None
217483,NaN,NaN,NaN,NaN,555716306,217484,PHL,Bontod-Maravilla Fish Sanctuary and Marine Res...,Bontod-Maravilla Fish Sanctuary and Marine Res...,Established,2005.0,Not Reported,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,0.000000,None
217484,NaN,NaN,NaN,NaN,555716297,217485,PHL,Bani Marine Protected Area,Bani Marine Protected Area,Established,2006.0,Not Reported,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,0.000000,None


In [93]:
pl_wd2 = pl_wd[pl_wd.places.notnull()]
len(pl_wd2)

19293

In [94]:
pl_wd2.head()

,index_left,GID_0,NAME_0,places,WDPA_PID,MOL_ID,ISO3,NAME,ORIG_NA,STATUS,STATUS_,GOV_TYP,DESIG,DESIG_E,DESIG_T,IUCN_CA,AREA_KM,geometry
1,1150.0,NZL,New Zealand,1155.0,307797,2,NZL,Ferndale,Ferndale,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.748492,"POLYGON ((173.97642 -41.19258, 173.97595 -41.1..."
2,1150.0,NZL,New Zealand,1155.0,307745,3,NZL,Broughton Bay,Broughton Bay,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.031907,"POLYGON ((173.95691 -41.20976, 173.95580 -41.2..."
3,1150.0,NZL,New Zealand,1155.0,307867,4,NZL,Kaipupu Point,Kaipupu Point,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.270855,"POLYGON ((174.00241 -41.27336, 174.00312 -41.2..."
6,1150.0,NZL,New Zealand,1155.0,555564797,7,NZL,Kenepuru Sound,Kenepuru Sound,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.228171,"POLYGON ((174.11032 -41.16784, 174.11051 -41.1..."
28,1151.0,NZL,New Zealand,1156.0,555565888,29,NZL,Robert Creek (Name to be confirmed),Robert Creek (Name to be confirmed),Designated,NaN,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,0.060156,"MULTIPOLYGON (((168.60922 -45.32145, 168.60800..."


In [96]:
pl_wd2 = pl_wd2.drop(columns={'index_left', 'geometry'})
pl_wd2

,GID_0,NAME_0,places,WDPA_PID,MOL_ID,ISO3,NAME,ORIG_NA,STATUS,STATUS_,GOV_TYP,DESIG,DESIG_E,DESIG_T,IUCN_CA,AREA_KM
1,NZL,New Zealand,1155.0,307797,2,NZL,Ferndale,Ferndale,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.748492
2,NZL,New Zealand,1155.0,307745,3,NZL,Broughton Bay,Broughton Bay,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.031907
3,NZL,New Zealand,1155.0,307867,4,NZL,Kaipupu Point,Kaipupu Point,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.270855
6,NZL,New Zealand,1155.0,555564797,7,NZL,Kenepuru Sound,Kenepuru Sound,Designated,NaN,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.228171
28,NZL,New Zealand,1156.0,555565888,29,NZL,Robert Creek (Name to be confirmed),Robert Creek (Name to be confirmed),Designated,NaN,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,0.060156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217366,MLI,Mali,968.0,23177,217367,DZA,Ahaggar,???????,Designated,1987.0,Federal or national ministry or agency,Parc Culturel,Cultural Park,National,OECM,542185.019063
217366,MLI,Mali,969.0,23177,217367,DZA,Ahaggar,???????,Designated,1987.0,Federal or national ministry or agency,Parc Culturel,Cultural Park,National,OECM,542185.019063
217395,DZA,Algeria,32.0,555705341,217396,DZA,Atlas Saharien,?????? ????????,Designated,2008.0,Federal or national ministry or agency,Parc Culturel,Cultural Park,National,OECM,88692.147941
217397,PHL,Philippines,1335.0,555716256,217398,PHL,Rafflesia schadenbergiana Critical Habitat,Rafflesia schadenbergiana Critical Habitat,Designated,2011.0,Joint governance,Critical Habitat,Critical Habitat,National,OECM,0.021364


In [97]:
pl_wd2.to_csv('/Users/sofia/Documents/HE_Data/PlacesHE/Outputs/WDPA_with_20places_updated.csv')

<a id='wdpa'></a>
### WDPA table with precalculated data (no geom)

In [107]:
wdpa_table.columns

Index(['Unnamed: 0', 'NAME', 'MOL_ID', 'AREA_KM2', 'WDPAID', 'DESIG',
       'DESIG_E', 'DESIG_T', 'IUCN_CA', 'ISO3', 'amphibians', 'birds',
       'mammals', 'reptiles', 'majority_land_cover_climate_regime',
       'land_cover_majority', 'climate_regime_majority', 'percent_irrigated',
       'percent_rainfed', 'percent_rangeland', 'percent_urban',
       'population_sum'],
      dtype='object')

In [109]:
wd.columns

Index(['WDPA_PID', 'MOL_ID', 'ISO3', 'NAME', 'ORIG_NA', 'STATUS', 'STATUS_',
       'GOV_TYP', 'DESIG', 'DESIG_E', 'DESIG_T', 'IUCN_CA', 'AREA_KM',
       'geometry'],
      dtype='object')

In [111]:
# Let's remove from wd the columns that are already in the wdpa table with contextual data
wd2 = wd.drop(columns={'NAME','AREA_KM', 'DESIG', 'DESIG_E', 'DESIG_T', 'IUCN_CA', 'ISO3'})
wd2

,WDPA_PID,MOL_ID,ORIG_NA,STATUS,STATUS_,GOV_TYP,geometry
0,310492,1,Boulder Beach / WWF Block,Designated,NaN,Federal or national ministry or agency,"MULTIPOLYGON (((170.62746 -45.89520, 170.62824..."
1,307797,2,Ferndale,Designated,NaN,Federal or national ministry or agency,"POLYGON ((173.97642 -41.19258, 173.97595 -41.1..."
2,307745,3,Broughton Bay,Designated,NaN,Federal or national ministry or agency,"POLYGON ((173.95691 -41.20976, 173.95580 -41.2..."
3,307867,4,Kaipupu Point,Designated,NaN,Federal or national ministry or agency,"POLYGON ((174.00241 -41.27336, 174.00312 -41.2..."
4,303963,5,Catlins Conservation Park,Designated,NaN,Federal or national ministry or agency,"MULTIPOLYGON (((169.30645 -46.57949, 169.30754..."
...,...,...,...,...,...,...,...
217481,555716517,217482,Sablayan Fish Sanctuary,Established,2000.0,Not Reported,None
217482,555716527,217483,Gurion Marine Sanctuary,Established,2002.0,Not Reported,None
217483,555716306,217484,Bontod-Maravilla Fish Sanctuary and Marine Res...,Established,2005.0,Not Reported,None
217484,555716297,217485,Bani Marine Protected Area,Established,2006.0,Not Reported,None


In [116]:
# Merge the new required fields (WDPA_PID, ORIG_NA, STATUS, STATUS_ and GOV_TYP)
wdpa_table2 = pd.merge(wdpa_table, wd2, on='MOL_ID')
wdpa_table2

,Unnamed: 0,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,...,percent_rainfed,percent_rangeland,percent_urban,population_sum,WDPA_PID,ORIG_NA,STATUS,STATUS_,GOV_TYP,geometry
0,0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,...,0.0,100.00,0.0,2.110001,310492,Boulder Beach / WWF Block,Designated,NaN,Federal or national ministry or agency,"MULTIPOLYGON (((170.62746 -45.89520, 170.62824..."
1,1,Ferndale,2,0.748492,307797.0,Scenic Reserve,Scenic Reserve,National,III,NZL,...,0.0,0.00,0.0,1.315837,307797,Ferndale,Designated,NaN,Federal or national ministry or agency,"POLYGON ((173.97642 -41.19258, 173.97595 -41.1..."
2,2,Broughton Bay,3,0.031907,307745.0,Scenic Reserve,Scenic Reserve,National,III,NZL,...,0.0,0.00,0.0,0.000000,307745,Broughton Bay,Designated,NaN,Federal or national ministry or agency,"POLYGON ((173.95691 -41.20976, 173.95580 -41.2..."
3,3,Kaipupu Point,4,0.270855,307867.0,Scenic Reserve,Scenic Reserve,National,III,NZL,...,0.0,100.00,0.0,0.000000,307867,Kaipupu Point,Designated,NaN,Federal or national ministry or agency,"POLYGON ((174.00241 -41.27336, 174.00312 -41.2..."
4,4,Catlins Conservation Park,5,8.412168,303963.0,Stewardship Area,Stewardship Area,National,III,NZL,...,0.0,83.33,0.0,3.103363,303963,Catlins Conservation Park,Designated,NaN,Federal or national ministry or agency,"MULTIPOLYGON (((169.30645 -46.57949, 169.30754..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217481,217481,Sablayan Fish Sanctuary,217482,0.000000,555716517.0,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,PHL,...,0.0,0.00,0.0,0.000000,555716517,Sablayan Fish Sanctuary,Established,2000.0,Not Reported,None
217482,217482,Gurion Marine Sanctuary,217483,0.000000,555716527.0,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,PHL,...,0.0,0.00,0.0,0.000000,555716527,Gurion Marine Sanctuary,Established,2002.0,Not Reported,None
217483,217483,Bontod-Maravilla Fish Sanctuary and Marine Res...,217484,0.000000,555716306.0,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,PHL,...,0.0,0.00,0.0,0.000000,555716306,Bontod-Maravilla Fish Sanctuary and Marine Res...,Established,2005.0,Not Reported,None
217484,217484,Bani Marine Protected Area,217485,0.000000,555716297.0,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,PHL,...,0.0,0.00,0.0,0.000000,555716297,Bani Marine Protected Area,Established,2006.0,Not Reported,None


In [124]:
wdpa_table2.columns

Index(['Unnamed: 0', 'NAME', 'MOL_ID', 'AREA_KM2', 'WDPAID', 'DESIG',
       'DESIG_E', 'DESIG_T', 'IUCN_CA', 'ISO3', 'amphibians', 'birds',
       'mammals', 'reptiles', 'majority_land_cover_climate_regime',
       'land_cover_majority', 'climate_regime_majority', 'percent_irrigated',
       'percent_rainfed', 'percent_rangeland', 'percent_urban',
       'population_sum', 'WDPA_PID', 'ORIG_NA', 'STATUS', 'STATUS_', 'GOV_TYP',
       'geometry'],
      dtype='object')

In [120]:
wdpa_table2.mammals[wdpa_table2.MOL_ID==1].values

array(['[{"SliceNumber":303.0,"per_global":0.0,"per_aoi":100.0},{"SliceNumber":2955.0,"per_global":0.0,"per_aoi":100.0},{"SliceNumber":4342.0,"per_global":0.01,"per_aoi":100.0}]'],
      dtype=object)

In [ ]:
wdpa_table2 = wdpa_table2.drop(columns={'Unnamed: 0', 'geometry'})

In [127]:
wdpa_table2.head()

,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,amphibians,...,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum,WDPA_PID,ORIG_NA,STATUS,STATUS_,GOV_TYP
0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,[],...,0.0,0.0,100.00,0.0,2.110001,310492,Boulder Beach / WWF Block,Designated,NaN,Federal or national ministry or agency
1,Ferndale,2,0.748492,307797.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...",...,0.0,0.0,0.00,0.0,1.315837,307797,Ferndale,Designated,NaN,Federal or national ministry or agency
2,Broughton Bay,3,0.031907,307745.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...",...,0.0,0.0,0.00,0.0,0.000000,307745,Broughton Bay,Designated,NaN,Federal or national ministry or agency
3,Kaipupu Point,4,0.270855,307867.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...",...,0.0,0.0,100.00,0.0,0.000000,307867,Kaipupu Point,Designated,NaN,Federal or national ministry or agency
4,Catlins Conservation Park,5,8.412168,303963.0,Stewardship Area,Stewardship Area,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...",...,0.0,0.0,83.33,0.0,3.103363,303963,Catlins Conservation Park,Designated,NaN,Federal or national ministry or agency


In [125]:
wdpa_table.columns

Index(['Unnamed: 0', 'NAME', 'MOL_ID', 'AREA_KM2', 'WDPAID', 'DESIG',
       'DESIG_E', 'DESIG_T', 'IUCN_CA', 'ISO3', 'amphibians', 'birds',
       'mammals', 'reptiles', 'majority_land_cover_climate_regime',
       'land_cover_majority', 'climate_regime_majority', 'percent_irrigated',
       'percent_rainfed', 'percent_rangeland', 'percent_urban',
       'population_sum'],
      dtype='object')

In [128]:
wdpa_table2.to_csv('/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Outputs/wdpa_precal_with_contextual_updated.csv')

---
<a id='publish'></a>
## Publish layers with API

In [36]:
# Run function with the file we want to add
csv_file = '/Users/sofia/Documents/HE_Data/Precalculated/gadm0/wdpa_with_gadm0_updated_copy.csv'
csvName = 'wdpa_with_gadm0_updated_copy'
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)
aol_folder_name = '#6 precalculated data'
group_id='46783c33c294485abe83f5e899c2f21f'
publishCSVasFS(csv_file,csvName,gis,aol_folder_name,table='yes',sharing='groups')

Service name wdpa_with_gadm0_updated_copy is available
Moving service wdpa_with_gadm0_updated_copy to #6 precalculated data in ArcGIS Online...
Service wdpa_with_gadm0_updated_copy has been moved to #6 precalculated data in ArcGIS Online
not a public layer, for this layer to be used it has to be public or the urls have to be whitelisted
wdpa_with_gadm0_updated_copy's protection against deletion : True


'3a59243e2c80412e8b7969e71206e344'

In [ ]:
# # Run function with the file we want to add
# csv_file = '/Users/sofia/Documents/HE_Data/Precalculated/gadm1/wdpa_with_gadm1_updated.csv'
# csvName = 'wdpa_with_gadm1_updated'
# gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)
# aol_folder_name = '#6 precalculated data'
# group_id='46783c33c294485abe83f5e899c2f21f'
# publishCSVasFS(csv_file,csvName,gis,aol_folder_name,table='yes',sharing='groups')

In [ ]:
# # Run function with the file we want to add
# csv_file = '/Users/sofia/Documents/HE_Data/PlacesHE/Outputs/WDPA_with_20places_updated.csv'
# csvName = 'wdpa_with_20places_updated'
# gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)
# aol_folder_name = '#6 precalculated data'
# group_id='46783c33c294485abe83f5e899c2f21f'
# publishCSVasFS(csv_file,csvName,gis,aol_folder_name,table='yes',sharing='groups')

Service name wdpa_with_20places_updated is available
{'message': "Service name 'data' already exists for 'IkktFdUAcY3WrH25'"}


In [ ]:
# # Run function with the file we want to add
# csv_file = '/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Outputs/wdpa_precal_with_contextual_updated.csv'
# csvName = 'wdpa_precal_with_contextual_updated'
# gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)
# aol_folder_name = '#6 precalculated data'
# group_id='46783c33c294485abe83f5e899c2f21f'
# publishCSVasFS(csv_file,csvName,gis,aol_folder_name,table='yes',sharing='groups')